In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
full_df = pd.read_csv('data/affcon_final.csv')

In [3]:
full_df.head()

,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,affcon_gamemove,affcon_reasoning,affcon_rapport,affcon_shareinformation,Input.speaker,...,Input.relative_message_index,Input.year,Input.game_score_speaker,Input.game_score_receiver,Input.game_score_delta,Input.deception_quadrant,Input.num_words,Input.num_characters,Input.sno,Input.sno1
0,Game7-turkey-austria,Train,Game7-turkey-austria-9,197,Im moving my fleet to Alb not for Greece but f...,1,1,1,1,austria-Game7,...,9,1901,3,3,0,Straightforward,12,56,32,16.0
1,Game10-england-russia,Train,Game10-england-russia-36,722,If you take action in the spring to fight Aust...,1,1,1,1,england-Game10,...,36,1905,7,7,0,Straightforward,28,150,100,12.0
2,Game6-england-germany,Train,Game6-england-germany-86,469,This is to help prevent the ottoman advance.,0,1,0,1,england-Game6,...,86,1909,6,9,3,Straightforward,8,44,30,5.0
3,Game11-austria-italy,Validation,Game11-austria-italy-5,45,"And yes I would like peace on our front, I cou...",1,1,1,0,austria-Game11,...,5,1901,3,3,0,Straightforward,31,132,46,22.0
4,Game6-germany-russia,Train,Game6-germany-russia-3,37,"Just to be clear, England is not necessarily a...",1,0,0,1,russia-Game6,...,3,1901,4,5,1,Cassandra,27,148,63,21.0


In [4]:
# X = full_df['Input.full_text'].to_list()
# y = full_df['affcon_rapport'].tolist()

X = full_df['Input.full_text']
y = full_df['affcon_rapport']

le = LabelEncoder() # this can convert our categories into labels, make sure you don't have NaNs or Nulls in your data first
y = le.fit_transform(y)
print(y.shape)

# we reshape 
y = y.reshape(-1,1) # the -1 allows it to have whatever number went in there
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(15738,)
(15738, 1)
(12590,)
(12590, 1)
(3148,)
(3148, 1)


In [5]:
max_chars = 0
max_words = 0

for i in range(len(X)):
    word_no = len(X[i].split(" "))
    char_no = len(X[i])
    if max_words < word_no:
        max_words = word_no
    if max_chars < char_no:
        max_chars = char_no

In [6]:
print(max_chars, max_words)

532 100


In [7]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

sequences = tok.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences,maxlen=max_len)
# X_train = sequence.pad_sequences(sequences)

In [48]:
Inp = Input(name='inputs',shape=[max_len])
x = Embedding(max_words,50,input_length=max_len)(Inp)
x = LSTM(32,name='LSTM_01')(x)
x = Dropout(0.5,name='Dropout')(x)
x = Dense(8,activation='relu',name='Dense_01')(x)
# x = Dropout(0.5,name='Dropout')(x)
out = Dense(1,activation='sigmoid', name='output')(x)

In [49]:
model = Model(inputs=Inp,outputs=out)

In [50]:
# model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [51]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 220)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 220, 50)           50000     
_________________________________________________________________
LSTM_01 (LSTM)               (None, 32)                10624     
_________________________________________________________________
Dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
Dense_01 (Dense)             (None, 8)                 264       
_________________________________________________________________
output (Dense)               (None, 1)                 9         
Total params: 60,897
Trainable params: 60,897
Non-trainable params: 0
_________________________________________________

In [52]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

In [53]:
# model.fit(X_train,y_train,
#           batch_size=128,
#           epochs=10,
#           validation_split=0.2,
#           callbacks=[early_stop])

model.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2)

Epoch 1/15
79/79 [==============================] - 2s 20ms/step - loss: 0.6692 - accuracy: 0.6038 - val_loss: 0.6620 - val_accuracy: 0.6044
Epoch 2/15
79/79 [==============================] - 1s 16ms/step - loss: 0.6503 - accuracy: 0.6045 - val_loss: 0.6539 - val_accuracy: 0.6044
Epoch 3/15
79/79 [==============================] - 1s 15ms/step - loss: 0.6361 - accuracy: 0.6089 - val_loss: 0.6541 - val_accuracy: 0.5977
Epoch 4/15
79/79 [==============================] - 1s 17ms/step - loss: 0.6273 - accuracy: 0.6172 - val_loss: 0.6590 - val_accuracy: 0.5631
Epoch 5/15
79/79 [==============================] - 1s 16ms/step - loss: 0.6183 - accuracy: 0.6356 - val_loss: 0.6725 - val_accuracy: 0.5802
Epoch 6/15
79/79 [==============================] - 1s 16ms/step - loss: 0.6139 - accuracy: 0.6442 - val_loss: 0.6716 - val_accuracy: 0.5794
Epoch 7/15
79/79 [==============================] - 1s 16ms/step - loss: 0.6075 - accuracy: 0.6490 - val_loss: 0.6807 - val_accuracy: 0.5759
Epoch 8/15
79

In [25]:
model = Sequential(name="CNN_with_embeddings")
model.add(Embedding(max_words, 50, input_length=max_len))
model.add(Conv1D(filters=16, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer= 'adam',
              metrics=['accuracy'])

model.summary()

Model: "CNN_with_embeddings"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 220, 50)           50000     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 217, 16)           3216      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 108, 16)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1728)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1728)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                17290     
_________________________________________________________________
dense_7 (Dense)              (None, 1)         

In [26]:
model.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2)

Epoch 1/15
79/79 [==============================] - 1s 11ms/step - loss: 0.6715 - accuracy: 0.5999 - val_loss: 0.6600 - val_accuracy: 0.6180
Epoch 2/15
79/79 [==============================] - 0s 6ms/step - loss: 0.6546 - accuracy: 0.6017 - val_loss: 0.6454 - val_accuracy: 0.6180
Epoch 3/15
79/79 [==============================] - 0s 6ms/step - loss: 0.6346 - accuracy: 0.6099 - val_loss: 0.6463 - val_accuracy: 0.6136
Epoch 4/15
79/79 [==============================] - 1s 7ms/step - loss: 0.6172 - accuracy: 0.6339 - val_loss: 0.6507 - val_accuracy: 0.6104
Epoch 5/15
79/79 [==============================] - 0s 6ms/step - loss: 0.5968 - accuracy: 0.6646 - val_loss: 0.6592 - val_accuracy: 0.5981
Epoch 6/15
79/79 [==============================] - 1s 7ms/step - loss: 0.5709 - accuracy: 0.6952 - val_loss: 0.6752 - val_accuracy: 0.5902
Epoch 7/15
79/79 [==============================] - 1s 7ms/step - loss: 0.5419 - accuracy: 0.7224 - val_loss: 0.6931 - val_accuracy: 0.5822
Epoch 8/15
79/79 [=

In [27]:
test_sequences = tok.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(test_sequences,maxlen=max_len)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [28]:
model.evaluate(X_test,y_test)

99/99 [==============================] - 0s 2ms/step - loss: 0.9314 - accuracy: 0.5499


[0.9313653707504272, 0.5498729348182678]